In [1]:
import pandas as pd
import numpy as np
import spacy

In [ ]:
news = pd.read_csv('../data/newsbank_articles.csv')

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
text ="""Netflix, Oracle and other big technology stocks lifted Wall Street Wednesday as their profits pile higher and excitement builds around the moneymaking prospects of artificial intelligence. The S&P 500 rose 0.6% and came close to its all-time closing high set early last month. The Dow Jones Industrial Average added 130 points, or 0.3%, and the Nasdaq composite climbed 1.3%. The gains came even though most U.S. stocks fell under the weight of another crank higher for Treasury yields in the bond market. The smaller stocks in the Russell 2000 index lost 0.6%, for example, and roughly two out of every three stocks in the S&P 500 sank. Gains for big, influential stocks were more than enough to make up for it. Netflix helped lead the way after it said live events like football games and a Mike Tyson-Jake Paul fight helped it add nearly 19 million subscribers during the latest quarter. It also reported stronger profit than analysts expected and said it’s raising subscription prices in the United States and other countries. Netflix jumped 9.7%. The streaming giant joined a lengthening list of companies that have topped analysts’ profit expectations for the end of 2024. Such results support their stock prices and counteract the downward push they’ve felt from rising Treasury yields, which can peel investors away from stocks. The rise in yields, caused in part by worries about stubborn inflation and the U.S. government’s swelling debt, had knocked down stocks and halted the record-breaking run that had carried them through 2024, at least briefly. Travelers climbed 3.2% after also topping analysts’ expectations for profit in the latest quarter. The insurer said gains for its investments and growth in net written premiums helped it overcome losses created by Hurricane Milton, which plowed into Florida’s Gulf coast in October, and other catastrophes. Procter & Gamble rose 1.9% after the company behind Charmin, Crest and Pampers reported stronger profit and revenue for the latest quarter than expected. It also said it’s sticking with its financial forecasts for the full fiscal year, even though rising commodity costs and the stronger value of the U.S. dollar against other currencies are hurting its results. Some of the market’s most forceful pushes upward came from AI-related companies. Oracle added 6.8% following its 7.2% rise the day before, ahead of the expected announcement that came late Tuesday about Stargate, a joint venture the White House said will start building out data centers and the electricity generation needed for the further development of AI in Texas. The partnership formed by Oracle, OpenAI and SoftBank will invest up to $500 billion. SoftBank Group Corp.’s stock in Tokyo rose 10.6%. Other AI-related stocks also climbed, furthering their already fantastic run. Nvidia, the company whose chips are powering much of the move into AI, rose 4.4%. Its stock is above $147 after sitting below $18 just two years ago. They helped offset a 3.4% drop for Textron. It reported a better profit for the latest quarter than analysts expected, but its revenue fell short of forecasts. The company behind Bell helicopters and Cessna airplanes also gave a forecast for profit for this upcoming year that fell short of some analysts’ expectations. All told, the S&P 500 rose 37.13 points to 6,086.37. The Dow Jones Industrial Average gained 130.92 to 44,156.73 and the Nasdaq composite jumped 252.56 to 20,009.34. In stock markets abroad, indexes were mixed in Europe and Asia. In the bond market, the 10-year Treasury yield rose to 4.60% from 4.57% late Tuesday. It had largely been regressing since an encouraging update on inflation last week, but it’s still well above where it was in September, when it was below 3.65%. Strategists at BlackRock Investment Institute say they expect rates to stay higher for longer for a range of reasons, including an aging workforce and high levels of debt for the U.S. government. “Investors are demanding more compensation for the risk of holding long-term bonds, driving the term premium to a decade high,” according to the strategists led by Jean Boivin. For stocks to remain resilient and rise despite the bite of high interest rates, companies will need to keep delivering solid profit growth, and Boivin said he sees encouraging signals. In the cryptocurrency market, where prices have surged on hopes President Donald Trump will make Washington friendlier to the industry, bitcoin was sitting just above $104,000. It had set a record above $109,000 on Monday. Some sourness is lingering after Trump and his wife launched meme coins, which critics said looked like an unseemly cash grab."""

In [5]:
spy_constituents = pd.read_excel('../data/sp_500_constituents.xlsx')
spy_constituents.head()

,PERMNO,Company Name,Ticker,SP500 Start,SP500 End,Search Keywords
0,10104,ORACLE CORP,ORCL,"Aug. 3, 1989","Dec. 31, 2024","ORCL, Oracle, Oracle Corp"
1,10107,MICROSOFT CORP,MSFT,1994-06-07 00:00:00,"Dec. 31, 2024","MSFT, Microsoft, Microsoft Corp"
2,10138,T ROWE PRICE GROUP INC,TROW,"Oct. 13, 1999","Dec. 31, 2024","T Rowe Price, T Rowe Price Group Inc, TROW, T ..."
3,10145,HONEYWELL INTERNATIONAL INC,HON,"Dec. 31, 1925","Dec. 31, 2024","HON, Honeywell, Honeywell International Inc"
4,10516,ARCHER DANIELS MIDLAND CO,ADM,1981-07-30 00:00:00,"Dec. 31, 2024","ADM, Archer Daniels Midland, Archer Daniels Mi..."


In [6]:
patterns = []
for index, row in spy_constituents.iterrows():
    for item in row['Search Keywords'].split(', '):
        temp_dict = {}
        temp_dict['label'] = "ORG"
        temp_dict['pattern'] = item
        patterns.append(temp_dict)


In [7]:
ruler = nlp.add_pipe("entity_ruler", before="ner", config={"overwrite_ents": True})
ruler.add_patterns(patterns)

In [8]:
doc = nlp(text)
found_financials = []
for ent in doc.ents:
    if ent.label_ == 'ORG':
        found_financials.append(ent.text)

In [9]:
found_financials = list(set(found_financials))
print(found_financials)

['Procter & Gamble', 'Hurricane Milton', 'SoftBank', 'Stargate', 'Travelers', 'meme coins', 'Pampers', 'BlackRock Investment Institute', 'Treasury', 'Bell', 'Netflix', 'Crest', 'Nvidia', 'SoftBank Group Corp.’s', 'Oracle', 'Cessna', 'Nasdaq', 'the White House', 'Textron']


In [ ]:
from transformers import pipeline

# --- Stage 2: Setup the ABSA Model (MODIFIED LINE) ---
# Load the amorpha/FinABSA model from Hugging Face
absa_classifier = pipeline("text-classification", model="amphora/FinABSA-Longer")

# --- Run the Pipeline (No Change) ---
print(f"Analyzing text: \"{text}\"\n")

# --- Classify Sentiment for Each Aspect (No Change) ---
results = []
for aspect in found_financials:
    # The pipeline function handles the specific input formatting for the model
    result = absa_classifier(text, text_pairs=aspect)
    sentiment = result[0]['label']
    score = result[0]['score']
    results.append({"aspect": aspect, "sentiment": sentiment, "score": f"{score:.2f}"})

# --- Display the Results (No Change) ---
for res in results:
    print(f"Aspect: '{res['aspect']}' -> Sentiment: {res['sentiment']} (Score: {res['score']})")

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at amphora/FinABSA and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Analyzing text: "Netflix, Oracle and other big technology stocks lifted Wall Street Wednesday as their profits pile higher and excitement builds around the moneymaking prospects of artificial intelligence. The S&P 500 rose 0.6% and came close to its all-time closing high set early last month. The Dow Jones Industrial Average added 130 points, or 0.3%, and the Nasdaq composite climbed 1.3%. The gains came even though most U.S. stocks fell under the weight of another crank higher for Treasury yields in the bond market. The smaller stocks in the Russell 2000 index lost 0.6%, for example, and roughly two out of every three stocks in the S&P 500 sank. Gains for big, influential stocks were more than enough to make up for it. Netflix helped lead the way after it said live events like football games and a Mike Tyson-Jake Paul fight helped it add nearly 19 million subscribers during the latest quarter. It also reported stronger profit than analysts expected and said it’s raising subscription p

TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'text_pairs'